In [1]:
%load_ext autoreload

In [2]:
import os
import sys
import torch
import numpy as np
import torchsummary

from pytf_utils import utils
from btcdet.models import build_network
from btcdet.config import cfg, cfg_from_list, cfg_from_yaml_file, log_config_to_file
from btcdet.datasets import build_dataloader
from btcdet.utils import common_utils
from btcdet.models import load_data_to_gpu


%autoreload 1
%aimport pytf_utils.utils

In [3]:
ROOT_DIR = utils.get_git_root(os.getcwd())

In [4]:
ckpt_file = os.path.join(ROOT_DIR, 'BtcDet', 'output', 'kitti_car', 'ckpt', 'checkpoint_epoch_1.pth')
log_file = os.path.join(ROOT_DIR, 'BtcDet', 'output', 'kitti_car', 'eval',
                        'eval_all_default', 'default', 'log_eval_20220412-171054.txt')
cfg_file = os.path.join(ROOT_DIR, 'cfgs', 'model_configs', 'btcdet_kitti_car.yaml')
cfg_from_yaml_file(cfg_file, cfg);
logger = common_utils.create_logger(log_file, rank=cfg.LOCAL_RANK)

/home/messi/alvaro/3D-perception/cfgs/model_configs/btcdet_kitti_car.yaml


In [5]:
batch_size = 2

In [6]:
test_set, test_loader, sampler = build_dataloader(
    dataset_cfg=cfg.DATA_CONFIG,
    class_names=cfg.CLASS_NAMES,
    batch_size=batch_size,
    dist=False, workers=4, training=False)

self.db_infos dict_keys(['Car'])
cur_cfg.NAME mask_points_and_boxes_outside_range
cur_cfg.NAME shuffle_points
cur_cfg.NAME transform_points_to_sphere_voxels
cur_cfg.NAME det_transform_points_to_voxels
self.dataset_cfg.INFO_PATH {'train': ['kitti_infos_train.pkl'], 'test': ['kitti_infos_val.pkl']}


In [7]:
model = build_network(model_cfg=cfg.MODEL, num_class=len(cfg.CLASS_NAMES), dataset=test_set)
model.cuda();
model.eval()

self.sparse_shape [  9 157 209]
input_channels 4
OccHead num_class 1
OccHead input_channels 32
self.stride !!!!!!!! 1 1
is_softmax: True
self.all_voxel_centers torch.Size([9, 157, 209, 3])
self.sparse_shape [  41 1600 1408]
input_channels 6
!!!local_grid_size [ 2  4 12]
!!!dims tensor([4.8000, 1.6000, 1.6000], device='cuda:0')
self.clamp_max None


/home/messi/anaconda3/envs/btcdet/lib/python3.9/site-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /opt/conda/conda-bld/pytorch_1639180487213/work/aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
/home/messi/alvaro/3D-perception/BtcDet/btcdet/models/occ_pnt/occ_training_targets/occ_targets_template.py:27: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  /opt/conda/conda-bld/pytorch_1639180487213/work/torch/csrc/utils/tensor_new.cpp:201.)
  self.point_origin_tensor = torch.as_tensor([point_cloud_range[:3]], dtype=torch.float32, device="cuda")


BtcNet(
  (occ_modules): Module(
    (occ_targets): OccTargets3D(
      (fix_conv_2dzy): Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    )
    (vfe): MeanVFE()
    (backbone_3d): VoxelBackBoneDeconv(
      (conv1): SparseSequential(
        (0): SparseSequential(
          (0): SparseConv3d()
          (1): BatchNorm1d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU()
        )
      )
      (conv2): SparseSequential(
        (0): SparseSequential(
          (0): SparseConv3d()
          (1): BatchNorm1d(32, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU()
        )
        (1): SparseSequential(
          (0): SubMConv3d()
          (1): BatchNorm1d(32, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU()
        )
      )
      (conv3): SparseSequential(
        (0): SparseSequential(
          (0): SparseConv3d()
          (1): BatchNorm1d(

In [8]:
model.load_params_from_file(ckpt_file, logger, to_cpu=False)

2022-04-12 20:16:48,060   INFO  ==> Loading  parameters from checkpoint /home/messi/alvaro/3D-perception/BtcDet/output/kitti_car/ckpt/checkpoint_epoch_1.pth to GPU
2022-04-12 20:16:48,919   INFO  ==> Done (loaded 399/399)


In [54]:
for i, batch_dict in enumerate(test_loader):
    load_data_to_gpu(batch_dict)
    break
    #
    # flops_model = add_flops_counting_methods(model)
    # flops_model.__batch_counter__ = 1
    # flops_model.eval()
    # flops_model.start_flops_count(ost=sys.stdout, verbose=True,
    #                               ignore_list=[])
    # macs, params = flops_model.compute_average_flops_cost()
    # print('{:<30}  {:<8}'.format('Computational complexity: ', macs))
    # print('{:<30}  {:<8}'.format('Number of parameters: ', params))

    with torch.no_grad():
        model.global_step = torch.tensor(i, dtype=torch.int64)
        pred_dicts, ret_dict, tb_dict, pc_dict = model(batch_dict)

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [15]:
i, batch_dict = next(enumerate(test_loader))
load_data_to_gpu(batch_dict)

In [13]:
%%timeit
i, batch_dict = next(enumerate(test_loader))
load_data_to_gpu(batch_dict)
with torch.no_grad():
    model.global_step = torch.tensor(i, dtype=torch.int64)
    pred_dicts, ret_dict, tb_dict, pc_dict = model(batch_dict)

502 ms ± 4.13 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [15]:
%%timeit
i, batch_dict = next(enumerate(test_loader))
load_data_to_gpu(batch_dict)

316 ms ± 6.93 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
